In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
from google.colab import files

uploaded = files.upload()


KeyboardInterrupt: 

In [7]:
import pandas as pd
df = pd.read_csv("/content/arbolado-publico-lineal-2017-2018.csv")
df.head()

/tmp/ipython-input-1922475503.py:2: DtypeWarning: Columns (2,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/arbolado-publico-lineal-2017-2018.csv")


,long,lat,nro_registro,tipo_activ,comuna,manzana,calle_nombre,calle_altura,calle_chapa,direccion_normalizada,ubicacion,nombre_cientifico,ancho_acera,estado_plantera,ubicacion_plantera,nivel_plantera,diametro_altura_pecho,altura_arbol
0,-58.378563,-34.594902,26779,Lineal,1,NaN,Esmeralda,1000.0,1120.0,ESMERALDA 1120,NaN,Tipuana tipu,5.5,Ocupada,Regular,A nivel,88.0,34.0
1,-58.381532,-34.592319,30887,Lineal,1,NaN,Pellegrini Carlos,1300.0,1345.0,"PELLEGRINI, CARLOS 1345",Exacta,Peltophorum dubium,4.5,Ocupada,Regular,Elevada,6.0,5.0
2,-58.379103,-34.591939,30904,Lineal,1,NaN,Arroyo,800.0,848.0,ARROYO 848,Exacta,Fraxinus pennsylvanica,4,Ocupada,Regular,A nivel,7.0,6.0
3,-58.380710,-34.591548,31336,Lineal,1,NaN,Arroyo,900.0,932.0,ARROYO 932,LD,Fraxinus pennsylvanica,NaN,Ocupada,Regular,A nivel,9.0,29.0
4,-58.380710,-34.591548,31337,Lineal,1,NaN,Arroyo,900.0,932.0,ARROYO 932,LA,Jacaranda mimosifolia,NaN,Ocupada,Regular,A nivel,13.0,8.0


In [9]:
import pandas as pd
import sqlite3
import plotly.express as px

In [11]:
# Cargar el CSV que subiste
df = pd.read_csv("arbolado.csv")

# Ver primeras filas
df.head()

/tmp/ipython-input-2616229881.py:2: DtypeWarning: Columns (2,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("arbolado.csv")


,long,lat,nro_registro,tipo_activ,comuna,manzana,calle_nombre,calle_altura,calle_chapa,direccion_normalizada,ubicacion,nombre_cientifico,ancho_acera,estado_plantera,ubicacion_plantera,nivel_plantera,diametro_altura_pecho,altura_arbol
0,-58.378563,-34.594902,26779,Lineal,1,NaN,Esmeralda,1000.0,1120.0,ESMERALDA 1120,NaN,Tipuana tipu,5.5,Ocupada,Regular,A nivel,88.0,34.0
1,-58.381532,-34.592319,30887,Lineal,1,NaN,Pellegrini Carlos,1300.0,1345.0,"PELLEGRINI, CARLOS 1345",Exacta,Peltophorum dubium,4.5,Ocupada,Regular,Elevada,6.0,5.0
2,-58.379103,-34.591939,30904,Lineal,1,NaN,Arroyo,800.0,848.0,ARROYO 848,Exacta,Fraxinus pennsylvanica,4,Ocupada,Regular,A nivel,7.0,6.0
3,-58.380710,-34.591548,31336,Lineal,1,NaN,Arroyo,900.0,932.0,ARROYO 932,LD,Fraxinus pennsylvanica,NaN,Ocupada,Regular,A nivel,9.0,29.0
4,-58.380710,-34.591548,31337,Lineal,1,NaN,Arroyo,900.0,932.0,ARROYO 932,LA,Jacaranda mimosifolia,NaN,Ocupada,Regular,A nivel,13.0,8.0


In [13]:
# Reducimos a las columnas más importantes
df_reducido = df[['nro_registro', 'comuna', 'calle_nombre', 'calle_altura',
                  'nombre_cientifico', 'diametro_altura_pecho',
                  'altura_arbol', 'lat', 'long']]

df_reducido.head()

,nro_registro,comuna,calle_nombre,calle_altura,nombre_cientifico,diametro_altura_pecho,altura_arbol,lat,long
0,26779,1,Esmeralda,1000.0,Tipuana tipu,88.0,34.0,-34.594902,-58.378563
1,30887,1,Pellegrini Carlos,1300.0,Peltophorum dubium,6.0,5.0,-34.592319,-58.381532
2,30904,1,Arroyo,800.0,Fraxinus pennsylvanica,7.0,6.0,-34.591939,-58.379103
3,31336,1,Arroyo,900.0,Fraxinus pennsylvanica,9.0,29.0,-34.591548,-58.380710
4,31337,1,Arroyo,900.0,Jacaranda mimosifolia,13.0,8.0,-34.591548,-58.380710


In [15]:
# Crear base de datos en memoria (puede ser archivo .db también)
conn = sqlite3.connect("arboles.db")
cursor = conn.cursor()

# Crear tabla
cursor.execute("""
CREATE TABLE IF NOT EXISTS arboles (
    nro_registro INTEGER,
    comuna INTEGER,
    calle_nombre TEXT,
    calle_altura INTEGER,
    nombre_cientifico TEXT,
    diametro_altura_pecho INTEGER,
    altura_arbol INTEGER,
    lat REAL,
    long REAL
)
""")

# Insertar los datos
df_reducido.to_sql("arboles", conn, if_exists="replace", index=False)

370180

In [17]:
# Ejemplo: cantidad de árboles por comuna
query = "SELECT comuna, COUNT(*) as total_arboles FROM arboles GROUP BY comuna"
result = pd.read_sql(query, conn)
result

,comuna,total_arboles
0,1,12723
1,2,7115
2,3,13320
3,4,32346
4,5,15224
5,6,15591
6,7,25608
7,8,21978
8,9,36405
9,10,33421


In [19]:
fig = px.bar(result, x="comuna", y="total_arboles",
             title="Cantidad de árboles por comuna en CABA")
fig.show()

In [21]:
query = """
SELECT nombre_cientifico, COUNT(*) as cantidad
FROM arboles
GROUP BY nombre_cientifico
ORDER BY cantidad DESC
LIMIT 10
"""
top_especies = pd.read_sql(query, conn)
top_especies

,nombre_cientifico,cantidad
0,Fraxinus pennsylvanica,133609
1,Platanus x acerifolia,32319
2,Ficus benjamina,22508
3,Tilia x moltkei,15605
4,Melia azedarach,14901
5,Jacaranda mimosifolia,13872
6,Lagerstroemia indica,11886
7,Fraxinus americana,10119
8,No identificado,9411
9,Tipuana tipu,9330


In [23]:
fig = px.bar(top_especies,
             x="nombre_cientifico",
             y="cantidad",
             title="Top 10 especies más comunes en CABA")
fig.show()

In [25]:
query = """
SELECT comuna, AVG(altura_arbol) as altura_promedio
FROM arboles
GROUP BY comuna
ORDER BY comuna
"""
altura_comunas = pd.read_sql(query, conn)
altura_comunas

,comuna,altura_promedio
0,1,8.620807
1,2,10.058966
2,3,9.362346
3,4,8.939791
4,5,9.915894
5,6,10.107078
6,7,8.072997
7,8,7.037197
8,9,7.380643
9,10,7.979431


In [27]:
fig = px.bar(altura_comunas,
             x="comuna",
             y="altura_promedio",
             title="Altura promedio de los árboles por comuna")
fig.show()

In [29]:
fig = px.histogram(df_reducido,
                   x="diametro_altura_pecho",
                   nbins=30,
                   title="Distribución de diámetros de troncos")
fig.show()

In [30]:
# Tomamos solo una muestra de 2000 filas para que no explote el mapa
df_muestra = df_reducido.sample(2000, random_state=42)

fig = px.scatter_mapbox(df_muestra,
                        lat="lat", lon="long",
                        hover_name="nombre_cientifico",
                        color="comuna",
                        zoom=10,
                        height=600)

fig.update_layout(mapbox_style="open-street-map")
fig.show()